# # A TikTok video classification (ref. Google-Coursera TikTok project)
# # Hyperparameter Tuning using GridSearchCV

In [1]:
# Import the necessary libraries
import pandas as pd

# Load the dataset
df = pd.read_csv('data/tiktok_dataset.csv')
# Display the first five rows of the dataset
print(df.head(5).to_string(), '\n')
# Display descriptive statistics
print(df.describe().to_string(), '\n')

# # Preprocess the dataset
# Drop the '#' and 'video_id' columns
df = df.drop(['#', 'video_id'], axis=1)
# Handling missing values
df = df.dropna()
# Handling duplicate values
df = df.drop_duplicates()
# Handling outliers
df = df[(df['video_download_count'] > 0) & (df['video_like_count'] > 0) & (df['video_comment_count'] > 0) & (df['video_share_count'] > 0)]

# Display descriptive statistics of the dataset after preprocessing
print(df.describe().to_string(), '\n')
# Check class balance
print(df['claim_status'].value_counts(), '\n')

In [1]:
# # Feature Engineering
# Extract the length of each video_transcription_text and add this as a column to the dataframe,
# so that it can be used as a potential feature in the model.
df['video_transcription_text_length'] = df['video_transcription_text'].apply(lambda x: len(x))
df.drop('video_transcription_text', axis=1, inplace=True)

# Calculate the average text_length for claims and opinions.
average_text_length_claim = df[df['claim_status'] == 'claim']['video_transcription_text_length'].mean()
average_text_length_opinion = df[df['claim_status'] == 'opinion']['video_transcription_text_length'].mean()

# Visualize the distribution of `text_length` for claims and opinions
# Create two histograms in one plot
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 5))
sns.histplot(df[df['claim_status'] == 'claim']['video_transcription_text_length'], color='blue', label='Claim', kde=True)
sns.histplot(df[df['claim_status'] == 'opinion']['video_transcription_text_length'], color='red', label='Opinion', kde=True)
plt.axvline(average_text_length_claim, color='blue', linestyle='dashed', linewidth=1)
plt.axvline(average_text_length_opinion, color='red', linestyle='dashed', linewidth=1)
plt.legend()
plt.title('Distribution of Text Length for Claims and Opinions')
plt.xlabel('Text Length')
plt.ylabel('Frequency')
plt.show()

# Encode target and categorical variables.
from sklearn.preprocessing import LabelEncoder

mapping = {}
# Encode the categorical variables
for col in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    mapping[col] = dict(zip(le.classes_, le.transform(le.classes_)))

# Display the mapping of the categorical variables
print(mapping, '\n')

# Display descriptive statistics of the dataset after preprocessing
print(df.describe(include='all').to_string(), '\n')

In [ ]:
# # Prepare the data for modeling
# Split the dataset into features and target variable
X = df.drop('claim_status', axis=1)
y = df['claim_status']

# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Display the shape of the training and testing sets
print('Training set shape:', X_train.shape, y_train.shape)
print('Testing set shape:', X_test.shape, y_test.shape, '\n')

# # Train the Random Forest model
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

# # GridSearchCV
from sklearn.model_selection import GridSearchCV

# scoring metrics
scoring = {'accuracy': 'accuracy',
           'precision': 'precision_weighted',
           'recall': 'recall_weighted',
           'f1': 'f1_weighted'}

# Create the hyperparameter grid - search space
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, 9],
    'max_samples': [0.5, 0.7, 0.9],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Instantiate the GridSearchCV object
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, 
                           verbose=2, scoring=scoring, refit='accuracy')

# Fit the GridSearchCV object
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_

# Display the best parameters
print('Best Parameters:', best_params, '\n')
# Display the best score
print('Best Score:', grid_search.best_score_, '\n')

In [ ]:
# # Evaluate the model

# Get the best model
best_model = grid_search.best_estimator_

# Predict the test set
y_pred = best_model.predict(X_test)

# Calculate the classification report
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
print('Classification Report:')
print(classification_report(y_test, y_pred), '\n')

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
presicion = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
print('Accuracy: {0}, Precision: {1}, Recall: {2}, F1 Score: {3}'.format(accuracy, presicion, recall, f1), '\n')

# Visualize the confusion matrix
import matplotlib.pyplot as plt
import seaborn as sns

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 5))
sns.set(font_scale=1.5)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()